In [1]:
import os
import json
import cv2
import pandas as pd

In [3]:
# json path
json_path = "./annotations/instances_default.json"

### json 파일 읽기

- import json이 필요함

- assert 조건, 조건실패시 메세지: 방어적 코딩을 위해 사용하는 조건문

    - 조건 실패시 에러 메세지와 조건실패시 메세지가 같이 나옴



In [9]:
with open(json_path, "r") as f:
    coco_info = json.load(f)

assert len(coco_info) > 0, "파일 읽기 실패"

### json파일에서 카테고리 정보 수집

In [10]:
categories = dict()
for category in coco_info['categories']:
    categories[category["id"]] = category["name"]
    
print("categories info >> ", categories)

categories info >>  {1: 'kiwi'}


### json파일에서 annotation 정보 수집

- dict()을 사용

    - image_id가 없을시 boxes정보와 categories정보를 입력

    - image_id가 존재할 경우, boxes정보와 categories정보를 추가시켜줌 

In [11]:
ann_info = dict()
for annotation in coco_info['annotations']:
    image_id = annotation["image_id"]
    bbox = annotation["bbox"]
    category_id = annotation["category_id"]

    if image_id not in ann_info:
        ann_info[image_id] = {
            "boxes": [bbox], "categories": [category_id]
        }
    else:
        ann_info[image_id]["boxes"].append(bbox)
        ann_info[image_id]["categories"].append(categories[category_id])

print("ann_info >> ", ann_info)

ann_info >>  {1: {'boxes': [[0.0, 29.22, 251.91, 191.06], [102.27, 199.74, 192.79, 107.54], [245.99, 0.68, 213.33, 100.93], [265.83, 57.76, 205.67, 182.7]], 'categories': [1, 'kiwi', 'kiwi', 'kiwi']}, 2: {'boxes': [[186.13, 23.4, 157.78, 154.28], [8.81, 8.74, 163.01, 192.33], [132.7, 162.7, 166.18, 149.02]], 'categories': [1, 'kiwi', 'kiwi']}, 3: {'boxes': [[391.27, 106.11, 356.75, 398.12], [32.72, 91.12, 368.14, 337.57]], 'categories': [1, 'kiwi']}, 4: {'boxes': [[419.42, 189.47, 289.31, 308.96], [166.27, 309.75, 367.93, 310.53], [43.63, 158.02, 382.08, 303.46]], 'categories': [1, 'kiwi', 'kiwi']}, 5: {'boxes': [[238.73, 84.38, 218.03, 249.48], [165.36, 189.73, 233.75, 222.22], [38.52, 142.03, 202.31, 204.41]], 'categories': [1, 'kiwi', 'kiwi']}, 6: {'boxes': [[73.13, 63.29, 57.0, 56.78], [114.76, 69.82, 55.44, 60.23], [110.54, 10.0, 45.26, 61.73], [57.79, 9.01, 52.75, 57.35], [49.16, 113.55, 71.35, 62.54], [0.24, 103.0, 49.11, 67.52], [7.72, 54.66, 55.82, 56.01], [16.35, 16.3, 46.81,

### 이미지 정보 출력

- json파일의 'images' 태그에서 width, height, id정보를 찾아옴

- 이미지를 불러온 후, json에서 불러온 박스정보를 토대로 bounding box를 이미지에 그려줌



In [27]:
file_list = []
x_list, y_list, w_list, h_list = [], [], [], []

for image_info in coco_info['images']:
    
    filename = image_info['file_name']
    width = image_info['width']
    height = image_info['height']
    img_id = image_info['id']
    print(filename, width, height, img_id)

    # 이미지 가져오기 위한 처리
    file_path = os.path.join("./images", filename)
    img = cv2.imread(file_path)
    try:
        annotation = ann_info[img_id]
    except KeyError:
        continue
    
    # box category
    for bbox, category in zip(annotation['boxes'], annotation['categories']):
        x1, y1, w, h = bbox
        rec_img = cv2.rectangle(img, (int(x1), int(y1)),
                                (int(x1+w), int(y1+h)), (225, 0, 255), 2)
        print(x1, y1, w, h)
    
    # cv2.imshow("test", rec_img)
    # cv2.waitKey(0)
    
    file_list.append(filename)
    x_list.append(x1)
    y_list.append(y1)
    w_list.append(w)
    h_list.append(h)
    
data_dict = {'fileName' : file_list,
'x1': x_list,
'y1': y_list,
'w' : w_list,
'h' : h_list
}
df = pd.DataFrame(data_dict)
df.to_csv('./', sep=',')
# print(df.head(10))

kiwi_1.jpg 500 332 1
0.0 29.22 251.91 191.06
102.27 199.74 192.79 107.54
245.99 0.68 213.33 100.93
265.83 57.76 205.67 182.7
kiwi_2.jpg 388 333 2
186.13 23.4 157.78 154.28
8.81 8.74 163.01 192.33
132.7 162.7 166.18 149.02
kiwi_3.jpg 764 572 3
391.27 106.11 356.75 398.12
32.72 91.12 368.14 337.57
kiwi_4.jpg 750 750 4
419.42 189.47 289.31 308.96
166.27 309.75 367.93 310.53
43.63 158.02 382.08 303.46
kiwi_5.jpg 500 500 5
238.73 84.38 218.03 249.48
165.36 189.73 233.75 222.22
38.52 142.03 202.31 204.41
kiwi_6.jpg 275 183 6
73.13 63.29 57.0 56.78
114.76 69.82 55.44 60.23
110.54 10.0 45.26 61.73
57.79 9.01 52.75 57.35
49.16 113.55 71.35 62.54
0.24 103.0 49.11 67.52
7.72 54.66 55.82 56.01
16.35 16.3 46.81 44.69
0.0 0.76 27.1 51.02
0.05 56.39 18.8 58.7
54.14 0.57 64.46 22.82
kiwi_7.jpg 1200 795 7
684.58 90.83 459.17 435.0
432.92 337.5 401.66 410.83
162.92 342.5 284.16 344.17
254.58 60.0 472.5 411.67


: 

### 이미지에 그려진 박스 정보를 xml파일로 저장

In [12]:
import xml.etree.ElementTree as ET

# 이미지 정보 출력
tree = ET.ElementTree()
root = ET.Element("annotations")
for i, image_info in enumerate(coco_info['images']):
    print('---------------------------')
    print('image info: ', image_info)
    
    # xml file save folder
    os.makedirs("./xml_folder/", exist_ok=True)
    xml_save_path = "./xml_folder/test.xml"

    filename = image_info['file_name']
    width = image_info['width']
    height = image_info['height']
    img_id = image_info['id']
    print(filename, width, height, img_id)
    xml_frame = ET.SubElement(
        root, "image", id=str(i), name=filename, width="%d" % width,
        height="%d" % height)

    # 이미지 가져오기 위한 처리
    file_path = os.path.join("./images", filename)
    img = cv2.imread(file_path)
    try:
        annotation = ann_info[img_id]
    except KeyError:
        continue
    # box category
    label_list = {1: "kiwi", 2: "apply"}

    for bbox, category in zip(annotation['boxes'], annotation['categories']):
        x1, y1, w, h = bbox
        print(filename, x1, y1, w, h)
        ET.SubElement(xml_frame, "box", label="Kiwi", occluded="0",
                      source="manual", x1=str(x1), y1=str(y1), w=str(w), h=str(h), z_order="0")
        rec_img = cv2.rectangle(img, (int(x1), int(y1)),
                                (int(x1+w), int(y1+h)), (225, 0, 255), 2)
        
    # cv2.imwrite()
    # cv2.imshow('Rectangle', rec_img)
    # cv2.waitKey(0)

    tree._setroot(root)
    tree.write(xml_save_path, encoding='utf-8')
    print("xml ok ")


---------------------------
image info:  {'id': 1, 'width': 500, 'height': 332, 'file_name': 'kiwi_1.jpg', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}
kiwi_1.jpg 500 332 1
kiwi_1.jpg 0.0 29.22 251.91 191.06
kiwi_1.jpg 102.27 199.74 192.79 107.54
kiwi_1.jpg 245.99 0.68 213.33 100.93
kiwi_1.jpg 265.83 57.76 205.67 182.7
xml ok 
---------------------------
image info:  {'id': 2, 'width': 388, 'height': 333, 'file_name': 'kiwi_2.jpg', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}
kiwi_2.jpg 388 333 2
kiwi_2.jpg 186.13 23.4 157.78 154.28
kiwi_2.jpg 8.81 8.74 163.01 192.33
kiwi_2.jpg 132.7 162.7 166.18 149.02
xml ok 
---------------------------
image info:  {'id': 3, 'width': 764, 'height': 572, 'file_name': 'kiwi_3.jpg', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}
kiwi_3.jpg 764 572 3
kiwi_3.jpg 391.27 106.11 356.75 398.12
kiwi_3.jpg 32.72 91.12 368.14 337.57
xml ok 
---------------------------
image info:  {'id': 4, 'wid